In [1]:
source vars.sh

# Containers on Milton

Presented by Michael Milton

# Introduction

## Acknowledgement of Country

This workshop is being held on the lands of the Wurundjeri People people and I wish to acknowledge them as Traditional Owners.

I would also like to pay my respects to their Elders, past and present, and Aboriginal Elders of other communities who may be here today.


## Outline

* Introduction
* Running Containers
* Break!
* Building Containers
* Hosting Containers

## Housekeeping

* Notebook is available at https://tinyurl.com/36p67eab
* Same content as slides
* Can use this to copy commands
* When you see the following, follow along:
<div class="alert alert-info" role="alert">
    ▶ Try this yourself! 
</div>

* The following cells are bash commands:

In [2]:
echo 'Hello!'

Hello!


## What are Containers?

* Single files
* Archives that contain other files (like `.tar` or `.zip`)
* Can execute commands *inside*
* Like lightweight virtual machines

## Why Containers?

* Reproducibility: ensure the analysis is insulated from changes in the outside system such as upgrades, environment variables etc.
    * e.g. you can compile a container with the exact R version, packages and data required to reproduce your analysis!
* Easy Usage: no need to install dependencies, use `make` etc, since they are all bundled. Sometimes users don't have permissions to install dependencies.
    * e.g. WEHI's AlphaFold
* Portability: the same `.sif` container can be run on any Linux machine with the same CPU architecture. No need to install using `apt`/`yum`/`conda` etc
    e.g. deploying a web application such as Shiny
* Maintainability: developers don't need to edit code to accommodate updates to language version etc
* OS Support: can be used to run old or new software that isn't compatible with the OS without breaking everything else.

[More information](https://apptainer.org/docs/user/main/introduction.html#use-cases)

## Why Apptainer?

Linux supports various container engines:

* Docker: most popular, but insecure on shared systems
* Podman: open source re-implementation of Docker. More secure
* Apptainer: different API from docker, designed to be HPC-first
* Singularity: old version of Apptainer

## What about Conda?

* Conda solves the same problem of software installation
* You can share conda environment files to increase reproducibility
* Conda is even usable with Nextflow
* However:
    * Sharing Conda environment files is like sharing a recipe rather than sharing your cookies
    * Installation can be quite slow due to resolving the dependencies each time
    * Not contained: using Conda still means your analysis is influenced by your environment, your operating system, and everything else you have installed
    * Certain platforms and engines only accept containers: Cromwell/WDL/terra.bio, AWS
    * If a tool is not in Conda, you can often compile a tool from scratch inside a container, which might be difficult or impossible with Conda
    * Doesn't solve legacy software issues
    * Fakeroot: you can pretend to be the root user inside a container

## Terminology

* **Image**: an archive containing a bundle of files. Read only.
* **Container**: an executing image. May be writeable.
* These two are often confused!
* **Host**: the machine running ("hosting") the container. 

# Running Containers

## Setup

```
ssh vc7-shared
salloc --partition interactive --time 4:00:00
git clone https://github.com/WEHI-ResearchComputing/ApptainerWorkshop.git
```

If you have access to `/vast/scratch`, you can make sure Apptainer doesn't fill up your home directory:
```
export APPTAINER_CACHEDIR=/vast/scratch/users/milton.m/apptainer_cache
export TMPDIR=/vast/scratch/users/milton.m/tmp
```

## Setting Up Apptainer

First we need to load apptainer. The most common way to load apptainer is as an [Environment Module](https://modules.readthedocs.io/en/latest/). On milton, the latest version we have access to is `1.1.0`.

<div class="alert alert-info" role="alert">
    ▶ Try this yourself!
</div>

In [3]:
module load apptainer/1.1.0

In [4]:
apptainer version

1.1.0


In [5]:
apptainer --help


Linux container platform optimized for High Performance Computing (HPC) and
Enterprise Performance Computing (EPC)

Usage:
  apptainer [global options...]

Description:
  Apptainer containers provide an application virtualization layer enabling
  mobility of compute via both application and environment portability. With
  Apptainer one is capable of building a root file system that runs on any
  other Linux system where Apptainer is installed.

Options:
      --build-config    use configuration needed for building containers
  -c, --config string   specify a configuration file (for root or
                        unprivileged installation only) (default
                        "/stornext/System/data/tools/apptainer/apptainer-1.1.0/etc/apptainer/apptainer.conf")
  -d, --debug           print debugging information (highest verbosity)
  -h, --help            help for apptainer
      --nocolor         print without color output (default False)
  -q, --quiet           suppress normal outpu

## Apptainer Run

You can run Apptainer images using the `apptainer run` command. This executes the "runscript" inside the container, which is the default executable:

In [6]:
apptainer run --help

Run the user-defined default command within a container

Usage:
  apptainer run [run options...] <container>

Description:
  This command will launch an Apptainer container and execute a runscript
  if one is defined for that container. The runscript is a metadata file within
  the container that contains shell commands. If the file is present (and
  executable) then this command will execute that file within the container
  automatically. All arguments following the container name will be passed
  directly to the runscript.

  apptainer run accepts the following container formats:

  *.sif               Singularity Image Format (SIF). Native to Singularity
                      (3.0+) and Apptainer (v1.0.0+)
  
  *.sqsh              SquashFS format.  Native to Singularity 2.4+

  *.img               ext3 format. Native to Singularity versions < 2.4.

  directory/          sandbox format. Directory containing a valid root file 
                      system and optionally Apptainer meta

                                      library://<hostname>/... URIs
      --no-init                       do NOT start shim process with --pid
      --no-mount strings              disable one or more 'mount xxx'
                                      options set in apptainer.conf and/or
                                      specify absolute destination path to
                                      disable a 'bind path' entry
      --no-privs                      drop all privileges from root user
                                      in container)
      --no-umask                      do not propagate umask to the
                                      container, set default 0022 umask
      --nv                            enable Nvidia support
      --nvccli                        use nvidia-container-cli for GPU
                                      setup (experimental)
      --oom-kill-disable              Disable OOM killer
  -o, --overlay strings               use an overlayFS imag

If you're working with an image you have downloaded, this will probably be a `.sif` file:

<div class="alert alert-info" role="alert">
    ▶ Try this yourself!
</div>

In [7]:
apptainer run WelcomeImage/hello.sif

@@@@@@@@@@@@@@@@@@@@@@@@@880GCCLLC@@@@CfLLCGG08@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@80CLfttttttt1f@@@@fiiiiiii1tfCG8@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@0CftttttttttttttL@@@@fi1111111iiii1tL0@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@0Ltttttttttttttttt1f@@@@fiiiii11111111iii1L0@@@@@@@@@@@@@@
@@@@@@@@@@@8GfttttttttttttttfLLCC0@@@@0CCLft1iiii111111iiitC8@@@@@@@@@@@
@@@@@@@@@8CttttttttttttfCG8@@@@@@@@@@@@@@@@@80GLt1ii111111ii1L8@@@@@@@@@
@@@@@@@@GtttttttttttLG8@@@@@@@@@@@@@@@@@@@@@@@@@@8Gf1ii111111i1C@@@@@@@@


@@@@@@8ftttttttttfG8@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@8Ctii111111it0@@@@@@
@@@@@GtttttttttfG@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@Gti111111i1G@@@@@
@@@@C1ttttttttC@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@C1i11111iiL@@@@
@@@@CLftttttf8@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@8tiiii1tfL8@@@
@@@@@@@80CfL@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ftLG8@@@@@@@
@@G08@@@@@@@@@@@@@@@@@@@@@@@@8LCCCCCCCCCCC@@@@@@@@@@@@@@@@@@@@@@@@@80G8@
@GtffLCG88@@@@@@@@@@@@@@@@@@@8:          L@@@@@@@@@@@@@@@@@@@@88GCLfttC@


@LfffffffG@@@@@@@@@@@@@@@@@@@@:          f@@@@@@@@@@@@@@@@@@@@Gttttffff8
0ffffffff8@@@@@@@@@@@@@@@@@@@:            f@@@@@@@@@@@@@@@@@@@8ffffffft0
GfffffffL@@@@@@@@@@@@@@@@@@@;              L@@@@@@@@@@@@@@@@@@@LtffffftC
GfffffffC@@@@@@@@@@@@@@@@@@i                C@@@@@@@@@@@@@@@@@@LtffffftC
GfffffffL@@@@@@@@@@@@@@@@@t        t.        G@@@@@@@@@@@@@@@@@LtffffftC
0ffffffff8@@@@@@@@@@@@@@@f        f@C        .0@@@@@@@@@@@@@@@@ffffffft0
@LfffffffG@@@@@@@@@@@@@@L        t@@@L        ,0@@@@@@@@@@@@@@Gtfffffff8


@GfffffffL@@@@@@@@@@@@@C        1@@@@@f        ,8@@@@@@@@@@@@@ffffffftC@
@@LfffffffC@@@@@@@@@@@G        i@@@@@@@f        :@@@@@@@@@@@@Ltfffffff8@
@@0ffffffffC@@@@@@@@@0.       ;@@@@@@@@@t        ;@@@@@@@@@@Ctfffffft0@@
@@@GffffffffC8@@@@@@8,       :@@@@@@@@@@@1        i@@@@@@@@LtfffffftG@@@
@@@@GffffffffL0@@@@8,       ;@@@@@@@@@@@@@1        1@@@@@0ftfffffftG@@@@
@@@@@0ffffffffC@@@@:        tG08@@@@@@@@80G,        t@@@@Ctffffftf0@@@@@
@@@@@@8CfffffC@@@@;            .,::::::,.            f@@@@CtffftL8@@@@@@


@@@@@@@@0LffL@@@@i                                    L@@@@LttfG@@@@@@@@
@@@@@@@@@@GC@@@@t        :.                  ,.        C@@@@LG@@@@@@@@@@
@@@@@@@@@@@@@@@f        C@80GLft1ii;;ii1tfLG0@0,        G@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@L       .C@@@@@@@@@@@@@@@@@@@@@@@0,       .G@@@@@@@@@@@@@@
@@@@@@@@@@@@8t        ;@@@@@@@@@@@@@@@@@@@@@@@@@i         L0@@@@@@@@@@@@
@@@@@@@@@@@fi,:::::::::if@@@@@@@@@@@@@@@@@@@@@81::::::::::,;t@@@@@@@@@@@


In [8]:
#apptainer run oras://ghcr.io/WEHI-ResearchComputing/hello:latest

Next, we will try to run a Docker container. This is the most common type of container you will likely find in the wild, since Apptainer is mostly only used on HPC. To run a Docker image in Apptainer, you will need to add `docker://` to the name of the image.

<div class="alert alert-info" role="alert">
    ▶ Try this yourself!
</div>

In [9]:
apptainer run docker://hello-world

INFO:    Using cached SIF image

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



Note that the Docker image was converted to `.sif` behind the scenes!

## Docker Hub

* Docker Hub is a common place to host Docker images
* However, it's not a great source for bioinformatics images (more on that later)
* You can search Docker Hub at: https://hub.docker.com/search?q=

<div class="alert alert-info" role="alert">
    ▶ Try this yourself!
    <ul>
        <li>See if you can find the official R Docker image. Then, <code>apptainer run</code> it.</li>
        <li>Tip: You might have to use some keywords.</li>
        <li>Tip 2: Remember to use the <code>docker://</code> prefix!</li>
        <li>Don't worry about warnings!</li>
    </ul>
</div>

In [10]:
echo 1 | apptainer run docker://r-base R --no-save

INFO:    Using cached SIF image

R version 4.2.2 (2022-10-31) -- "Innocent and Trusting"
Copyright (C) 2022 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> 1
[1] 1
> 


<div class="alert alert-success" role="alert"><code>apptainer run docker://r-base</code></div>

## Apptainer Exec

* Executes a custom command inside the container and **not** the runscript
* More likely to use this than `apptainer run`
* Required for many containers (such as biocontainers) that have no runscript

In [11]:
apptainer exec --help

Run a command within a container

Usage:
  apptainer exec [exec options...] <container> <command>

Description:
  apptainer exec supports the following formats:

  *.sif               Singularity Image Format (SIF). Native to Singularity
                      (3.0+) and Apptainer (v1.0.0+)
  
  *.sqsh              SquashFS format.  Native to Singularity 2.4+

  *.img               ext3 format. Native to Singularity versions < 2.4.

  directory/          sandbox format. Directory containing a valid root file 
                      system and optionally Apptainer meta-data.

  instance://*        A local running instance of a container. (See the instance
                      command group.)

  library://*         A SIF container hosted on a Library (no default)

  docker://*          A Docker/OCI container hosted on Docker Hub or another
                      OCI registry.

  shub://*            A container hosted on Singularity Hub.

  oras://*            A SIF container hosted on an O

      --no-privs                      drop all privileges from root user
                                      in container)
      --no-umask                      do not propagate umask to the
                                      container, set default 0022 umask
      --nv                            enable Nvidia support
      --nvccli                        use nvidia-container-cli for GPU
                                      setup (experimental)
      --oom-kill-disable              Disable OOM killer
  -o, --overlay strings               use an overlayFS image for
                                      persistent data storage or as
                                      read-only layer of container
      --passphrase                    prompt for an encryption passphrase
      --pem-path string               enter an path to a PEM formatted RSA
                                      key for an encrypted container
  -p, --pid                           run container in a new PID names

<div class="alert alert-info" role="alert">
    ▶ Try this yourself! 
</div>

In [12]:
apptainer exec WelcomeImage/hello.sif ls /opt

message.txt


In [13]:
 ls /opt

nvidia  quantum


## Filesystems

* **Unlike** Docker, and most other container engines, Apptainer automatically "mounts" parts of the HPC filesystem into each container by default
* This means that you can access some of your local files inside the container, along with the tools provided by the container

In [14]:
apptainer exec docker://ubuntu ls ~

INFO:    Using cached SIF image
ApptainerTutorial      Scrooge		       jobRegistry
GASAL2		       curated_annotation.rds  metadata.sqlite
HCAquery	       data		       ondemand
HcaBenchmarking        dials.find_spots.log    output
HcaIntegration	       files_for_michael.rds   scratch
HcaPython	       files_list.rds	       temp
IntegrationBenchmarks  hca.sqlite-journal      testdata
InterProScanData       igv		       wf-clone-validation
PythonIntegration      interproscan-5.51-85.0
R		       job.sh


In [15]:
ls ~

ApptainerTutorial       HCAquery                output
curated_annotation.rds  hca.sqlite-journal      PythonIntegration
data                    igv                     R
dials.find_spots.log    IntegrationBenchmarks   scratch
files_for_michael.rds   interproscan-5.51-85.0  Scrooge
files_list.rds          InterProScanData        temp
GASAL2                  jobRegistry             testdata
HcaBenchmarking         job.sh                  wf-clone-validation
HcaIntegration          metadata.sqlite
HcaPython               ondemand


## Sample Data

* In `./data` in the git repository is a sample fasta file
* This is an unknown protein that we will be investigating

<div class="alert alert-info" role="alert">
    ▶ Try this yourself! 
    <ul>
        <li>Find the official NCBI blast image on docker hub, then execute blastp to discover the identity of the protein</li>
        <li>Hint: I suggest using the following syntax: <code>blastp -query $INPUT_FILE -db refseq_select -remote -max_target_seqs 5</code></li>
</div>

In [16]:
apptainer exec docker://ncbi/blast blastp -db refseq_select -query ~/ApptainerWorkshop/data/test.fasta -remote -max_target_seqs 5

INFO:    Using cached SIF image
BLASTP 2.13.0+


Reference: Stephen F. Altschul, Thomas L. Madden, Alejandro A.
Schaffer, Jinghui Zhang, Zheng Zhang, Webb Miller, and David J.
Lipman (1997), "Gapped BLAST and PSI-BLAST: a new generation of
protein database search programs", Nucleic Acids Res. 25:3389-3402.


Reference for composition-based statistics: Alejandro A. Schaffer,
L. Aravind, Thomas L. Madden, Sergei Shavirin, John L. Spouge, Yuri
I. Wolf, Eugene V. Koonin, and Stephen F. Altschul (2001),
"Improving the accuracy of PSI-BLAST protein database searches with
composition-based statistics and other refinements", Nucleic Acids
Res. 29:2994-3005.



Database: RefSeq Select proteins
           43,770,235 sequences; 14,340,472,515 total letters

Error: [blastp] Failed to fetch sequences in batch mode


Query= sample1

Length=339

RID: RBHWAEXH013
                                                                      Score     E
Sequences producing significant alignments:               

Sbjct  293  ATIEARQRMAELAARNLIAVLKGEM--PPALV  322


>WP_011011434.1 glyoxylate reductase [Pyrococcus furiosus]
Length=336

 Score = 140 bits (354),  Expect = 7e-36, Method: Compositional matrix adjust.
 Identities = 106/338 (31%), Positives = 169/338 (50%), Gaps = 10/338 (3%)

Query  3    PNVDKAEAIPEATFSFLDVLFDLCVAKDMIPERQYVMLEKDREDQLMELMMDEIMDADT-  61
            P V    AIPE   + L+  F++ V ++     +  +LEK ++   +  M+ E +D +  
Sbjct  3    PKVFITRAIPENGINMLEEEFEVEVWEEEREIPREKLLEKVKDVDALVTMLSERIDQEVF  62

Query  62   EGANNIKIMSEKQSALCYIRQDVTTKPGILVSNGPDTLSNALANYTESMTKVVFGEVVPS  121
            E A  ++I++        I  +  T+ GI V+N PD L+NA A++  ++       VV  
Sbjct  63   ENAPRLRIVANYAVGYDNIDVEEATRRGIYVTNTPDVLTNATADHAFALLLATARHVVKG  122

Query  122  DKTYIVDLITYQWNKISFPENTLTEYGLSNKVLGILGMGRLIGKLADRYNTFNLRIMYFP  181
            DK   V    ++   I++       Y L  K +GI+G GR+   +A R   FN+RI+Y+ 
Sbjct  123  DK--FVRSGEWKRKGIAWHPKWFLGYELYGKTIGIVGFGRIGQAIARRAKGFNMRILYYS  180

Query  182  ADRKQAFECILCVSYSPFEEVL

## Interactive Sessions with Apptainer Shell

* `apptainer shell`
* Lets you run commands interactively inside a container
* Acts like `ssh`

In [18]:
cat /etc/os-release

NAME="CentOS Linux"
VERSION="7 (Core)"
ID="centos"
ID_LIKE="rhel fedora"
VERSION_ID="7"
PRETTY_NAME="CentOS Linux 7 (Core)"
ANSI_COLOR="0;31"
CPE_NAME="cpe:/o:centos:centos:7"
HOME_URL="https://www.centos.org/"
BUG_REPORT_URL="https://bugs.centos.org/"

CENTOS_MANTISBT_PROJECT="CentOS-7"
CENTOS_MANTISBT_PROJECT_VERSION="7"
REDHAT_SUPPORT_PRODUCT="centos"
REDHAT_SUPPORT_PRODUCT_VERSION="7"



<div class="alert alert-info" role="alert">
    ▶ Try this yourself! 
</div>

In [19]:
echo 'cat /etc/os-release' | apptainer shell docker://ubuntu

INFO:    Using cached SIF image
PRETTY_NAME="Ubuntu 22.04.1 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.1 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


## Comparison of Commands

* `exec`, `run`, and `shell` are all command variants that run containers
* They all have almost the same flags, and they all require an image as the main argument
* `apptainer run` executes the default command
* `apptainer exec` executes a custom command
* `apptainer shell` executes an interactive shell

## Filesystems - Writeable

* Outside of the mounted directories, containers are read-only by default. You need either the `--writable` or `--writable-tmpfs` flags to edit files
* Also, if you want to act as the root (administrator) user, you need `--fakeroot`

## Filesystems - Custom Mounts

* You can mount additional directories into the container using `--bind`
* You can use this flag with any apptainer command such as `exec`, `shell` etc
* `apptainer run docker://ubuntu --bind /vast/scratch/users/milton.m/` would make `/vast/scratch/users/milton.m/` accessible inside the container
*  `apptainer run docker://ubuntu --bind /vast/scratch/users/milton.m:/scratch` would make `/scratch` inside the container link to `/vast/scratch/users/milton.m` on milton
* This is equivalent to `-v` in docker (you will often see this in readmes)

In [20]:
apptainer exec docker://ubuntu ls /scratch

INFO:    Using cached SIF image
/usr/bin/ls: cannot access '/scratch': No such file or directory


: 2

## Aside: Order of Flags

* Apptainer *has* to be run with the following syntax: `apptainer exec --apptainer-flags image.sif command --command-flags`
* `apptainer exec image.sif --bind /vast/scratch` *will not work* because the bind command is after the image name

In [21]:
apptainer exec --bind /vast/scratch/users/milton.m:/scratch docker://ubuntu ls /scratch

INFO:    Using cached SIF image
alphafold_singularity  apptainer  cache  singularity  tmp


<div class="alert alert-info" role="alert">
    ▶ Try this yourself!
    <ul>
        <li>We want to run <a href="https://hub.docker.com/r/interpro/interproscan"><code>interproscan</code></a> on our fasta file.</li>
        <li>Interproscan expects certain reference data to be located in <code>/opt/interproscan</code> inside the container
        <li>This data is located in <code>/stornext/System/data/apps/sample-scripts/interproscan/interproscan-5.51-85.0/data</code> on milton</li>
        <li>You can run the tool as <code>interproscan.sh --input ~/ApptainerWorkshop/data/test.fasta --disable-precalc</code>.</li>
        <li>Results will be output to <code>test.fasta.tsv</code> in the current directory.</li>
    </ul>
</div>

In [22]:
apptainer exec \
    --bind /stornext/System/data/apps/sample-scripts/interproscan/interproscan-5.51-85.0/data:/opt/interproscan/data \
    --workdir $HOME \
    docker://interpro/interproscan \
    interproscan.sh \
    --applications Pfam \
    --input ~/ApptainerWorkshop/data/test.fasta \
    --disable-precalc

INFO:    Using cached SIF image
17/11/2022 15:35:25:967 Welcome to InterProScan-5.51-85.0
17/11/2022 15:35:25:970 Running InterProScan v5 in STANDALONE mode... on Linux
log4j:WARN No appenders could be found for logger (org.apache.activemq.broker.BrokerService).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.
17/11/2022 15:35:46:142 RunID: slurm-login02.hpc.wehi.edu.au_20221117_153545658_566l
17/11/2022 15:36:01:833 Loading file /home/users/allstaff/milton.m/ApptainerTutorial/data/test.fasta
17/11/2022 15:36:01:836 Running the following analyses:
[Pfam-33.1]
Pre-calculated match lookup service DISABLED.  Please wait for match calculations to complete...
17/11/2022 15:36:03:940 30% completed
17/11/2022 15:36:25:469 70% completed
17/11/2022 15:36:32:074 80% completed
17/11/2022 15:36:37:866 100% done:  InterProScan analyses completed 



In [23]:
cat test.fasta.tsv

sample1	b20fd131a682ede28053e1c9b627f90a	339	Pfam	PF02826	D-isomer specific 2-hydroxyacid dehydrogenase, NAD binding domain	140	294	1.3E-18	T	17-11-2022	IPR006140	D-isomer specific 2-hydroxyacid dehydrogenase, NAD-binding domain


## Biocontainers

* Docker images automatically built from bioconda
* Most bioinformatics tools are available
* Can search at https://bioconda.github.io/search.html?q=

![](media/biocontainers.png)

<div class="alert alert-info" role="alert">
    ▶ Try this yourself!
    <br/>
    <ul>
        <li>Try to find the latest biocontainer of the Diamond local aligner</li>
        <li>Make sure you include the tag!</li>
    </ul>
</div>

<div class="alert alert-success" role="alert"><code>apptainer shell docker://quay.io/biocontainers/diamond:2.0.15--hb97b32f_1</code></div>

# Break

## Building Containers

* You can build a container using `apptainer build image.sif recipe.def`
* `recipe.def` is a container definition file that contains instructions for building

In [24]:
apptainer build --help

Build an Apptainer image

Usage:
  apptainer build [local options...] <IMAGE PATH> <BUILD SPEC>

Description:

  IMAGE PATH:

  When Apptainer builds the container, output can be one of a few formats:

      default:    The compressed Apptainer read only image format (default)
      sandbox:    This is a read-write container within a directory structure

  note: It is a common workflow to use the "sandbox" mode for development of the
  container, and then build it as a default Apptainer image for production
  use. The default format is immutable.

  BUILD SPEC:

  The build spec target is a definition (def) file, local image, or URI that can 
  be used to create an Apptainer container. Several different local target
  formats exist:

      def file  : This is a recipe for building a container (examples below)
      directory:  A directory structure containing a (ch)root file system
      image:      A local image on your machine (will convert to sif if
                  it is legacy fo

<div class="alert alert-info" role="alert">
    ▶ Try this yourself!
</div>

In [25]:
cd WelcomeImage
apptainer build image.sif hello.def
apptainer run image.sif
cd ..

INFO:    User not listed in /etc/subuid, trying root-mapped namespace
INFO:    The %post section will be run under fakeroot
INFO:    Starting build...
Getting image source signatures
Copying blob e96e057aae67 skipped: already exists  
Copying config 134185395f done  
Writing manifest to image destination
Storing signatures
2022/11/17 16:36:45  info unpack layer: sha256:e96e057aae67380a4ddb16c337c5c3669d97fdff69ec537f02aa2cc30d814281
INFO:    Copying logo.txt to /opt/message.txt
INFO:    Adding runscript
INFO:    Creating SIF file...
INFO:    Build complete: image.sif
@@@@@@@@@@@@@@@@@@@@@@@@@880GCCLLC@@@@CfLLCGG08@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@80CLfttttttt1f@@@@fiiiiiii1tfCG8@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@0CftttttttttttttL@@@@fi1111111iiii1tL0@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@0Ltttttttttttttttt1f@@@@fiiiii11111111iii1L0@@@@@@@@@@@@@@
@@@@@@@@@@@8GfttttttttttttttfLLCC0@@@@0CCLft1iiii111111iiitC8@@@@@@@@@@@
@@@@@@@@@8CttttttttttttfCG8@@@@@@@@@@@@@@@@@80GLt1ii111111ii1

@@@@@@@@GtttttttttttLG8@@@@@@@@@@@@@@@@@@@@@@@@@@8Gf1ii111111i1C@@@@@@@@
@@@@@@8ftttttttttfG8@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@8Ctii111111it0@@@@@@
@@@@@GtttttttttfG@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@Gti111111i1G@@@@@
@@@@C1ttttttttC@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@C1i11111iiL@@@@
@@@@CLftttttf8@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@8tiiii1tfL8@@@
@@@@@@@80CfL@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ftLG8@@@@@@@
@@G08@@@@@@@@@@@@@@@@@@@@@@@@8LCCCCCCCCCCC@@@@@@@@@@@@@@@@@@@@@@@@@80G8@


@GtffLCG88@@@@@@@@@@@@@@@@@@@8:          L@@@@@@@@@@@@@@@@@@@@88GCLfttC@
@LfffffffG@@@@@@@@@@@@@@@@@@@@:          f@@@@@@@@@@@@@@@@@@@@Gttttffff8
0ffffffff8@@@@@@@@@@@@@@@@@@@:            f@@@@@@@@@@@@@@@@@@@8ffffffft0
GfffffffL@@@@@@@@@@@@@@@@@@@;              L@@@@@@@@@@@@@@@@@@@LtffffftC
GfffffffC@@@@@@@@@@@@@@@@@@i                C@@@@@@@@@@@@@@@@@@LtffffftC
GfffffffL@@@@@@@@@@@@@@@@@t        t.        G@@@@@@@@@@@@@@@@@LtffffftC
0ffffffff8@@@@@@@@@@@@@@@f        f@C        .0@@@@@@@@@@@@@@@@ffffffft0


@LfffffffG@@@@@@@@@@@@@@L        t@@@L        ,0@@@@@@@@@@@@@@Gtfffffff8
@GfffffffL@@@@@@@@@@@@@C        1@@@@@f        ,8@@@@@@@@@@@@@ffffffftC@
@@LfffffffC@@@@@@@@@@@G        i@@@@@@@f        :@@@@@@@@@@@@Ltfffffff8@
@@0ffffffffC@@@@@@@@@0.       ;@@@@@@@@@t        ;@@@@@@@@@@Ctfffffft0@@
@@@GffffffffC8@@@@@@8,       :@@@@@@@@@@@1        i@@@@@@@@LtfffffftG@@@
@@@@GffffffffL0@@@@8,       ;@@@@@@@@@@@@@1        1@@@@@0ftfffffftG@@@@
@@@@@0ffffffffC@@@@:        tG08@@@@@@@@80G,        t@@@@Ctffffftf0@@@@@


@@@@@@8CfffffC@@@@;            .,::::::,.            f@@@@CtffftL8@@@@@@
@@@@@@@@0LffL@@@@i                                    L@@@@LttfG@@@@@@@@
@@@@@@@@@@GC@@@@t        :.                  ,.        C@@@@LG@@@@@@@@@@
@@@@@@@@@@@@@@@f        C@80GLft1ii;;ii1tfLG0@0,        G@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@L       .C@@@@@@@@@@@@@@@@@@@@@@@0,       .G@@@@@@@@@@@@@@
@@@@@@@@@@@@8t        ;@@@@@@@@@@@@@@@@@@@@@@@@@i         L0@@@@@@@@@@@@
@@@@@@@@@@@fi,:::::::::if@@@@@@@@@@@@@@@@@@@@@81::::::::::,;t@@@@@@@@@@@


## Build Definitions

* Def files are text files
* Consist of mandatory header, followed by optional `%sections`
* `%files` copies files from the host into the container
* `%environment` sets environment variables
* `%post` runs commands inside the container
* `%runscript` sets the runscript
* [More detail](https://apptainer.org/docs/user/main/definition_files.html)

```
Bootstrap: docker
From: ubuntu:18.04

%files
    /file1
    /file1 /opt

%environment
    export LISTEN_PORT=12345
    export LC_ALL=C

%post
    apt-get update && apt-get install -y netcat
    NOW=`date`
    echo "export NOW=\"${NOW}\"" >> $APPTAINER_ENVIRONMENT

%runscript
    echo "Container was created $NOW"
    echo "Arguments received: $*"
    exec echo "$@"

%test
    grep -q NAME=\"Ubuntu\" /etc/os-release
    if [ $? -eq 0 ]; then
        echo "Container base is Ubuntu as expected."
    else
        echo "Container base is not Ubuntu."
        exit 1
    fi

%labels
    Author alice
    Version v0.0.1

%help
    This is a demo container used to illustrate a def file that uses all
    supported sections.
```

In [32]:
cat WelcomeImage/hello.def

Bootstrap: docker
From: ubuntu

%files
    logo.txt /opt/message.txt

%runscript
    cat /opt/message.txt


<div class="alert alert-info" role="alert">
    ▶ Try this yourself!
    <br/>
    <ul>
        <li>Try to make a container that uses the R <a href="https://github.com/sckott/cowsay#install">cowsay</a> package</li>
        <li>You will have to install the appropriate R package, and then run the <code>say()</code> function using the user input</li>
        <li>You can use the <code>$*</code> bash variable to access user input</li>
        <li>The <code>Rscript</code> command can be used to run R commands</li>
    </ul>
</div>

In [33]:
cat cowsay.def

Bootstrap: docker
From: r-base

%post
    Rscript -e 'install.packages("cowsay")'

%runscript
    Rscript -e "cowsay::say('$*')"


In [34]:
apptainer build cowsay.sif cowsay.def

INFO:    User not listed in /etc/subuid, trying root-mapped namespace
INFO:    The %post section will be run under fakeroot
INFO:    Starting build...
Getting image source signatures
Copying blob ae8780930e7e skipped: already exists  
Copying blob ebbe46658ae1 skipped: already exists  
Copying blob b6e2154a522a skipped: already exists  
Copying blob 36a417257f63 skipped: already exists  
Copying blob ae8780930e7e skipped: already exists  
Copying blob ebbe46658ae1 skipped: already exists  
Copying blob b6e2154a522a skipped: already exists  
Copying blob 36a417257f63 skipped: already exists  
Copying blob 48f11b798771 skipped: already exists  
Copying blob ced6bc7d0fb6 skipped: already exists  
Copying config 935885ce10 done  
Writing manifest to image destination
Storing signatures
2022/11/17 16:38:39  info unpack layer: sha256:ebbe46658ae1eddd748e3222cbc9dd7109f9fd7f279a4b2f9d6a32d0a58b4c16
2022/11/17 16:38:41  info unpack layer: sha256:ae8780930e7e7b18116589a863916682a85c45bec3c738da

In [35]:
apptainer run cowsay.sif catfact


 -------------- 
A cat has the power to sometimes heal themselves by purring. A domestic cat's purr has a frequency of between 25 and 150 Hertz, which happens to be the frequency at which muscles and bones best grow and repair themselves.  
 --------------
    \
      \
        \
            |\___/|
          ==) ^Y^ (==
            \  ^  /
             )=*=(
            /     \
            |     |
           /| | | |\
           \| | |_|/\
      jgs  //_// ___/
               \_)
  

## Hosting Containers

* You can share your apptainer images as files that you upload to a storage service or keep on a server
* You can also upload images to container repositories such as GitHub, so people can run them directly from the internet

In [36]:
apptainer push --help

Upload image to the provided URI

Usage:
  apptainer push [push options...] <image> <URI>

Description:
  The 'push' command allows you to upload a SIF container to a given
  URI.  Supported URIs include:

  library:
      library://user/collection/container[:tag]

  oras:
      oras://registry/namespace/image:tag


  NOTE: It's always good practice to sign your containers before
  pushing them to the library. An auth token is required to push to the library,
  so you may need to configure it first with 'apptainer remote'.

Options:
  -U, --allow-unsigned       do not require a signed container image
  -D, --description string   description for container image (library:// only)
  -h, --help                 help for push
      --library string       the library to push to
      --no-https             use http instead of https for docker://
                             oras:// and library://<hostname>/... URIs


Examples:
  To Library
  $ apptainer push /home/user/my.sif library://user/c

### Create a Token

In order to push a container to GitHub, you will need to generate an access token for Apptainer to use

<div class="alert alert-info" role="alert">
    ▶ Try this yourself!
</div>

* Visit https://github.com/settings/tokens
* "Generate a new token (classic)"
* Check `write:packages`
* Click "Generate token"

## Login

In [37]:
apptainer remote login --username $GITHUB_USERNAME --password $GITHUB_TOKEN docker://ghcr.io

INFO:    Token stored in /home/users/allstaff/milton.m/.apptainer/remote.yaml


In [38]:
apptainer push cowsay.sif oras://ghcr.io/$GITHUB_USERNAME/cowsay

INFO:    No tag or digest found, using default: latest
INFO:    Upload complete


You can now view the package at [`https://github.com/$GITHUB_USERNAME?tab=packages`](https://github.com/multimeric?tab=packages)

In [40]:
apptainer run oras://ghcr.io/$GITHUB_USERNAME/cowsay:latest catfact

INFO:    Using cached SIF image

 -------------- 
All cats have claws, and all except the cheetah sheath them when at rest. 
 --------------
    \
      \
        \
            |\___/|
          ==) ^Y^ (==
            \  ^  /
             )=*=(
            /     \
            |     |
           /| | | |\
           \| | |_|/\
      jgs  //_// ___/
               \_)
  